## Carregando dataset

---


In [9]:
import pandas as pd

df = pd.read_csv("https://dados-ml-pln.s3-sa-east-1.amazonaws.com/produtos.csv", delimiter=";", encoding='utf-8')
df.describe()

,nome,descricao,categoria
count,4080,2916,4080
unique,3696,2460,4
top,Bonecos Brinquedo Vingadores 30 Cm Com Luz E...,JOGO ORIGINAL. NOVO. LACRADO. PRONTA ENTREGA. ...,livro
freq,20,39,1020


In [11]:
# limpa nulos
df.dropna(inplace=True)

# concatena nome e descrição do produto
df["texto"] = df['nome'] + " " + df['descricao']

df.texto[0]

' O Hobbit - 7ª Ed. 2013  Produto NovoBilbo Bolseiro é um hobbit que leva uma vida confortável e sem ambições. Mas seu contentamento é perturbado quando Gandalf, o mago, e uma companhia de anões batem à sua porta e levam-no para uma expedição. Eles têm um plano para roubar o tesouro guardado por Smaug, o Magnífico, um grande e perigoso dragão. Bilbo reluta muito em participar da aventura, mas acaba surpreendendo até a si mesmo com sua esperteza e sua habilidade como ladrão!CaracterísticasAutor: Tolkien, J. R. R.Peso: 0.44I.S.B.N.: 9788578277109Altura: 20.000000Largura: 13.000000Profundidade: 1.000000Número de Páginas: 328Idioma: PortuguêsAcabamento: BrochuraNúmero da edição: 7Ano da edição: 2013'

In [12]:
df.describe()

,nome,descricao,categoria,texto
count,2916,2916,2916,2916
unique,2584,2460,4,2646
top,Boneco Dragon Ball Z Son Gokou,JOGO ORIGINAL. NOVO. LACRADO. PRONTA ENTREGA. ...,livro,Boneco Dragon Ball Z Son Gokou Produto novo
freq,20,39,838,20


Tratamento e vetorização

In [13]:
# 1 - Contagem de termos com unigrama
from sklearn.feature_extraction.text import CountVectorizer

stops = nltk.corpus.stopwords.words('portuguese')

vect = CountVectorizer(ngram_range=(1,1))
#vect = CountVectorizer(ngram_range=(1,1), stop_words=stops)
vect.fit(df.texto)
text_vect = vect.transform(df.texto)

In [ ]:
# 2 - TF-IDF com unigrama - referencia
from sklearn.feature_extraction.text import TfidfVectorizer

stops = nltk.corpus.stopwords.words('portuguese')

vect = TfidfVectorizer(ngram_range=(1,1), use_idf=True)
#vect = TfidfVectorizer(ngram_range=(1,1), use_idf=True, stop_words=stops)
vect.fit(df.texto)
text_vect = vect.transform(df.texto)

In [ ]:
# 3 - TF-IDF com unigrama e aplicando o stemização RSLP
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

df['tokens'] = df.texto.apply(word_tokenize)

from nltk.stem.rslp import RSLPStemmer
import nltk
nltk.download('rslp')

rslp = RSLPStemmer()

def stem_pandas(line):  
  return ' '.join([rslp.stem(token) for token in line])

df['stemmer'] = df.tokens.apply(stem_pandas)
df.head(2)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

stops = nltk.corpus.stopwords.words('portuguese')

vect = TfidfVectorizer(ngram_range=(1,1), use_idf=True)
#vect = TfidfVectorizer(ngram_range=(1,1), use_idf=True, stop_words=stops)
#vect = TfidfVectorizer(ngram_range=(1,1), use_idf=False, stop_words=stops)
#vect = TfidfVectorizer(ngram_range=(1,2), use_idf=False, stop_words=stops, smooth_idf=False, norm='l1')

vect.fit(df.stemmer)
text_vect = vect.transform(df.stemmer)

Treinando e validando o modelo

In [14]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(
    text_vect, 
    df["categoria"],
    test_size = 0.3, 
    random_state = 42
)

from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(random_state=42)
tree.fit(X_train, y_train)

#tree.score(X_train, y_train)
tree.score(X_test, y_test)

0.9565714285714285

Exemplo 1.1 - 0.9565714285714285

Exemplo 1.2 - 0.9588571428571429

Exemplo 2.1 - 0.952

Exemplo 2.2 - 0.9554285714285714

Exemplo 3.1 - 0.9302857142857143

Exemplo 3.2 - 0.92

Exemplo 3.3 - 0.9291428571428572

Exemplo 3.4 - 0.9405714285714286

<br>

**Melhor modelo foi do exemplo 1.2: 0.9588571428571429**

In [15]:
from sklearn.metrics import accuracy_score

y_prediction = tree.predict(X_test)

accuracy = accuracy_score(y_prediction, y_test)

print(accuracy)

0.9565714285714285


In [16]:
texto = """
Maleta Exclusiva Ruby Rose Profissional Completa. 
"""

texto_vetor = vect.transform([texto])

In [17]:
tree.predict(texto_vetor)

array(['maquiagem'], dtype=object)

## Pipeline completo de treino e teste

Carregando os dados

In [1]:
import pandas as pd

df = pd.read_csv("https://dados-ml-pln.s3-sa-east-1.amazonaws.com/produtos.csv", delimiter=";", encoding='utf-8')
df.describe()

,nome,descricao,categoria
count,4080,2916,4080
unique,3696,2460,4
top,Bonecos Brinquedo Vingadores 30 Cm Com Luz E...,JOGO ORIGINAL. NOVO. LACRADO. PRONTA ENTREGA. ...,livro
freq,20,39,1020


Normalização

In [3]:
# limpa nulos
df.dropna(inplace=True)

# concatena nome e descrição do produto
df["texto"] = df['nome'] + " " + df['descricao']

Amostra treino e teste antes da vetorização

In [4]:
from sklearn.model_selection import train_test_split

# divisão da amostra entre treino e teste
df_train, df_test = train_test_split(
      df, 
      test_size = 0.3, 
      random_state = 42
  )

Tratamento e vetorização

In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
# 1 - Contagem de termos com unigrama
from sklearn.feature_extraction.text import CountVectorizer

stops = nltk.corpus.stopwords.words('portuguese')

vect = CountVectorizer(ngram_range=(1,1), stop_words=stops)
vect.fit(df_train.texto)
text_vect = vect.transform(df_train.texto)

Treinamento do modelo

In [7]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(random_state=42)
tree.fit(text_vect, df_train["categoria"])

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=42, splitter='best')

Validação do modelo

In [8]:
x_test = vect.transform(df_test.texto)
y_test = df_test["categoria"]

tree.score(x_test, y_test)

0.9588571428571429